In [1]:
import os
import subprocess
from subprocess import Popen, PIPE

In [2]:
def bash(cmd):
    process = Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
    stdout, stderr = process.communicate()

    print(stderr.decode('ascii'))
    print(stdout.decode('ascii'))

Barcode file creating

In [ ]:
path_to_raw_data = '' # write path to ypur data

In [ ]:
bash(f'ls {path_to_raw_data} > names.txt')

In [ ]:
with open(f'barcode_file.tsv', 'w') as file:
    with open('names.txt') as names:
        for name in names.readlines():
            name = name[:-1]
            if name.find('R1') != -1:
                sample_id = name.split('_R1')[0]
                name_R2 = name.replace('R1', 'R2')
                file.write(f'{sample_id}\ttggtatcaacgcagagtNNNNtNNNNtNNNN\t\t{path_to_raw_data}/{name}\t{path_to_raw_data}/{name_R2}\n')

# migec

In [ ]:
bash('migec CheckoutBatch -cute barcode_file.tsv checkout')

In [ ]:
bash('migec Histogram checkout/ histogram/')

Overseq heatmap 

In [ ]:
overseq = pd.read_csv(f"./histogram/overseq.txt", sep="\t", header=0)
plt.rcParams['figure.figsize'] = [10, 10] # change these numbers to change figure size
labels=overseq.loc[:,"1":"1024"]
overseq_scaled=overseq.loc[:,"1":"1024"].div(overseq.loc[:,"1":"1024"].max(axis=1), axis=0)
fig = sns.heatmap(overseq_scaled,yticklabels=overseq["#SAMPLE_ID"],annot=labels, fmt="d")
fig.get_figure().savefig('overseq.png')

Treshold 2 !!!

In [ ]:
bash('migec AssembleBatch --force-overseq 2 -c checkout/ histogram/ assemble/')

# mixcr

In [ ]:
os.mkdir('analyze')

MusMusculus!!!

In [ ]:
with open(f'assemble/assemble.log.txt') as file:
    file.readline()
    for line in file.readlines():
        line = line.split('\t')
        name = line[0]
        R1 = line[4]
        R2 = line[5]
        bash(f'mixcr analyze amplicon -s MusMusculus --starting-material rna --5-end no-v-primers \
                    --3-end c-primers --adapters no-adapters {R1} {R2} analyze/{name}')

# vdjtools

In [ ]:
os.mkdir('convert')

In [ ]:
bash('ls analyze > analyze_names.txt')

In [ ]:
# there you can add additional information that can be extracted from names
with open(f'analyze/metadata.txt', 'w') as metadata:
    metadata.write('file_name\tsample_id\n')
    with open(f'analyze_names.txt') as names:
        for name in names.readlines():
            if name.find('clonotypes.TRB.') != -1:
                name = name[:-1]
                sample_id = name.split('.clonotypes.')[0]
                metadata.write(f'{name}\t{sample_id}\n')

In [ ]:
bash('vdjtools Convert -S MiXcr -m analyze/metadata.txt convert/')

In [ ]:
os.mkdir('filtered')

In [ ]:
bash('vdjtools FilterNonFunctional -m convert/metadata.txt filtered/')

In [ ]:
bash('vdjtools CalcBasicStats -m filtered/metadata.txt filtered')